# FRM analysis

Reading the data and finding out Recency , Frequency and Monetory value

In [2]:
import pandas as pd
import numpy as np
df21=pd.read_csv('C:/Users/ASUS/Documents/Bank_transaction/9-month(inactive).csv')

df21=df21[['cid','amount','txn_date','Category','Sub_Category']]
df21['Frequency']=1
df21['last_txn_date'] = df21.groupby('cid')['txn_date'].transform('max')
df21['last_txn_date'] = pd.to_datetime(df21['last_txn_date'], infer_datetime_format=True)
df21['today_date']=pd.to_datetime("now")
df21['today_date'] = df21['today_date'].dt.date
df21["today_date"] = pd.to_datetime(df21["today_date"])

###(alternatively)
df21['today_date'] = pd.to_datetime('2020/9/13', format='%Y/%m/%d')
####

df21['diff_days'] = df21['today_date']-df21['last_txn_date']
df21['diff_days']=df21['diff_days'].dt.days
df21.drop(['txn_date','Sub_Category','Category','today_date'], axis='columns', inplace=True)
df22=df21.groupby(['cid','diff_days','last_txn_date'], as_index=False).agg({"amount" : "sum", "Frequency":"sum"})
df22

C:\Users\ASUS\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,8,9,12,14,15,18,19,50,51,52,53,54,55,56,58,59,60,61,65,67,68,70,75) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cid,diff_days,last_txn_date,amount,Frequency
0,2,197,2020-02-29,52732,34
1,3,205,2020-02-21,29789,6
2,5,370,2019-09-09,488177,90
3,7,408,2019-08-02,21340,19
4,8,350,2019-09-29,49008,19
...,...,...,...,...,...
10617,19995,357,2019-09-22,11543,7
10618,19997,220,2020-02-06,3989,11
10619,19998,249,2020-01-08,394,1
10620,19999,349,2019-09-30,56347,75


Binning the data for scoring further.

The following bins are selected as best distributions by analysing their parameters like mean, std, count, min, max separately in excel. 

In [4]:
## Binning ##

bins = [190, 221, 261, 311, 371, 465]
labels = [5,4,3,2,1]
df22['days_binned'] = pd.cut(df22['diff_days'], bins=bins, labels=labels)

bins = [0,50,200,350,500,2050]
labels = [1,2,3,4,5]
df22['Freq._binned'] = pd.cut(df22['Frequency'], bins=bins, labels=labels)

bins = [-1,9485,35539,77808,156389,10829155]
labels = [1,2,3,4,5]
df22['amount_binned'] = pd.cut(df22['amount'], bins=bins, labels=labels)

SCORING & Segmenting the customers is the main and final stage of FRM

In [5]:
## Scoring ##

df22['Bins']=df22['days_binned'].astype(str) + df22['Freq._binned'].astype(str) + df22['amount_binned'].astype(str)
df22['Average']=df22[['days_binned','Freq._binned','amount_binned']].mean(axis=1)
df22

,cid,diff_days,last_txn_date,amount,Frequency,days_binned,Freq._binned,amount_binned,Bins,Average
0,2,197,2020-02-29,52732,34,5,1,3,513,3.000000
1,3,205,2020-02-21,29789,6,5,1,2,512,2.666667
2,5,370,2019-09-09,488177,90,2,2,5,225,3.000000
3,7,408,2019-08-02,21340,19,1,1,2,112,1.333333
4,8,350,2019-09-29,49008,19,2,1,3,213,2.000000
...,...,...,...,...,...,...,...,...,...,...
10617,19995,357,2019-09-22,11543,7,2,1,2,212,1.666667
10618,19997,220,2020-02-06,3989,11,5,1,1,511,2.333333
10619,19998,249,2020-01-08,394,1,4,1,1,411,2.000000
10620,19999,349,2019-09-30,56347,75,2,2,3,223,2.333333
